In [5]:
import pandas as pd
# Load the existing Commons images metadata CSV file
existing_commons_images = pd.read_csv('commons_images.csv', na_filter=False, dtype = str)
# Get a list of the column headers from the existing Commons images metadata CSV file
existing_commons_images_columns = existing_commons_images.columns.tolist()
existing_commons_images = existing_commons_images.set_index('qid')

updated_manifest_metadata = pd.read_csv('commons_images_manifest_reupload.csv', na_filter=False, dtype = str)
updated_manifest_metadata = updated_manifest_metadata.set_index('qid')

works_metadata = pd.read_csv('../works_multiprop.csv', na_filter=False, dtype = str)
works_metadata = works_metadata.set_index('qid')

# Create a new dataframe for the statements to be deleted
statements_to_delete = pd.DataFrame(columns=['qid', 'iiif_manifest_uuid'])


In [2]:
# Extract a list of the Q IDs from the updated manifest metadata
updated_manifest_metadata_qids = updated_manifest_metadata.index.tolist()

# Find the rows of the existing Commons images metadata that don't have a matching Q ID in the updated manifest metadata
existing_commons_images_manifest_not_updated = existing_commons_images[~existing_commons_images.index.isin(updated_manifest_metadata_qids)]

# Save the dataframe to a CSV file
existing_commons_images_manifest_not_updated.to_csv('existing_commons_images_manifest_not_updated.csv')

In [6]:
# Make a list of the manifest URLs from the existing Commons images metadata
existing_images_manifest_urls_list = existing_commons_images['iiif_manifest'].tolist()

# Find the Q IDs of the rows in the updated manifest metadata whose manifest URLs don't match the manifest URLs in the existing Commons images metadata
updated_manifest_not_matched_qids_list = updated_manifest_metadata[~updated_manifest_metadata['iiif_manifest'].isin(existing_images_manifest_urls_list)].index.tolist()

# Find the iiif_manifest_uuids from the works_metadata dataframe for the Q IDs in the updated_manifest_not_matched_qids_list

# Create a dictionary of qids and iiif_manifest_uuids from the works_metadata dataframe for the Q Ids in the updated_manifest_not_matched_qids_list
updated_manifest_not_matched_qids_dict = works_metadata.loc[updated_manifest_not_matched_qids_list]['iiif_manifest_uuid'].to_dict()

# Create a dictionary of the iiif_manifest URLs from the existing commons metadata for the Q IDs in the updated_manifest_not_matched_qids_list
existing_manifest_not_matched_manifest_urls_dict = existing_commons_images.loc[updated_manifest_not_matched_qids_list]['iiif_manifest'].to_dict()

# Create a dictionary of the iiif_manifest URLs from the updated manifest metadata for the Q IDs in the updated_manifest_not_matched_qids_list
updated_manifest_not_matched_manifest_urls_dict = updated_manifest_metadata.loc[updated_manifest_not_matched_qids_list]['iiif_manifest'].to_dict()

# Create dataframes for the three dictionaries and merge them on the qid column
updated_manifest_not_matched_qids_df = pd.DataFrame.from_dict(updated_manifest_not_matched_qids_dict, orient='index', columns=['iiif_manifest_uuid'])
existing_manifest_not_matched_manifest_urls_df = pd.DataFrame.from_dict(existing_manifest_not_matched_manifest_urls_dict, orient='index', columns=['iiif_manifest'])
updated_manifest_not_matched_manifest_urls_df = pd.DataFrame.from_dict(updated_manifest_not_matched_manifest_urls_dict, orient='index', columns=['iiif_manifest'])
# Merge the dataframes on the qid column
updated_manifest_not_matched_df = updated_manifest_not_matched_qids_df.merge(existing_manifest_not_matched_manifest_urls_df, left_index=True, right_index=True)
updated_manifest_not_matched_df = updated_manifest_not_matched_df.merge(updated_manifest_not_matched_manifest_urls_df, left_index=True, right_index=True)

# Rename the iiif_manifest_x column to iiif_manifest_old
updated_manifest_not_matched_df = updated_manifest_not_matched_df.rename(columns={'iiif_manifest_x': 'iiif_manifest_old'})
# Rename the iiif_manifest_y column to iiif_manifest_new
updated_manifest_not_matched_df = updated_manifest_not_matched_df.rename(columns={'iiif_manifest_y': 'iiif_manifest_new'})

# Save the dataframe to a CSV file
updated_manifest_not_matched_df.to_csv('deletions.csv')

In [14]:
updated_manifest_not_matched_df.head()

,iiif_manifest_uuid,iiif_manifest_old,iiif_manifest_new
Q106577245,179B6F99-221A-4B14-BA4D-29BD3D4797E8,https://iiif-manifest.library.vanderbilt.edu/g...,https://iiif-manifest.library.vanderbilt.edu/g...
Q106577246,6FA0BD9C-2F3E-46C0-A591-6A9BC989A0AC,https://iiif-manifest.library.vanderbilt.edu/g...,https://iiif-manifest.library.vanderbilt.edu/g...
Q106577247,DDAF8F9A-5D1E-497F-AA5C-578CBE815A84,https://iiif-manifest.library.vanderbilt.edu/g...,https://iiif-manifest.library.vanderbilt.edu/g...
Q106577248,A8AE5A8C-0904-4D2E-9087-8994A719BA5D,https://iiif-manifest.library.vanderbilt.edu/g...,https://iiif-manifest.library.vanderbilt.edu/g...
Q106577249,57BB4E6F-AF7A-46E1-8FC8-D04315812369,https://iiif-manifest.library.vanderbilt.edu/g...,https://iiif-manifest.library.vanderbilt.edu/g...


In [7]:
# Replace the old manifest URLs in the works_metadata dataframe with the new manifest URLs and delete the iiif_manifest_uuid values for those rows
works_metadata.loc[updated_manifest_not_matched_qids_list, 'iiif_manifest'] = updated_manifest_not_matched_df['iiif_manifest_new']
works_metadata.loc[updated_manifest_not_matched_qids_list, 'iiif_manifest_uuid'] = ''

# Save the dataframe to a CSV file
works_metadata.to_csv('../works_multiprop.csv')

# Don't rerun this

After determining which manifests got skipped, generate a version of the source file to be run without any screening

In [2]:
import pandas as pd

# Load the existing images whose manifests didn't get updated
existing_commons_images_manifest_not_updated = pd.read_csv('existing_commons_images_manifest_not_updated.csv', na_filter=False, dtype = str)
existing_commons_images_manifest_not_updated = existing_commons_images_manifest_not_updated.set_index('qid')

# Load the source artwork_metadata.csv into a dataframe 
artwork_metadata = pd.read_csv('artwork_metadata.csv', na_filter=False, dtype = str)
artwork_metadata = artwork_metadata.set_index('qid')

# Create a new dataframe containing the source rows whose Q IDs are in the existing_commons_images_manifest_not_updated dataframe
artwork_metadata_manifest_not_updated = artwork_metadata.loc[existing_commons_images_manifest_not_updated.index]

# Save the dataframe to a CSV file
artwork_metadata_manifest_not_updated.to_csv('artwork_metadata_manifest_not_updated.csv')

# After running this, re-run commonstool.py to generate the new manifests and upload them